In [1]:
import time
import numpy

from scipy.optimize import fmin_l_bfgs_b
from scipy.spatial.distance import cdist

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

from matplotlib import pyplot

In [2]:
class DiscriminantANoyau:

    def __init__(self, lambda_, sigma):
        # Cette fonction est déjÃ  codée pour vous, vous n'avez qu'Ã  utiliser
        # les variables membres qu'elle définit dans les autres fonctions de
        # cette classe.
        # Lambda et sigma sont définis dans l'énoncé.
        self.lambda_ = lambda_
        self.sigma = sigma
        
    def fit(self, X, y):
        if set(y) == {0, 1}:
            y = (y * 2) - 1
        elif set(y) != {-1, 1}:
            raise Exception("The target must be {0,1} or {-1,1}")
        # Implémentez la fonction d'entraÃ®nement du classifieur, selon
        # les équations que vous avez développées dans votre rapport.
        
        # TODO Q3B
        # Vous devez écrire une fonction nommée evaluateFunc,
        # qui reÃ§oit un seul argument en paramÃ¨tre, qui correspond aux
        # valeurs des paramÃ¨tres pour lesquels on souhaite connaÃ®tre
        # l'erreur et le gradient d'erreur pour chaque paramÃ¨tre.
        # Cette fonction sera appelée Ã  répétition par l'optimiseur
        # de scipy, qui l'utilisera pour minimiser l'erreur et obtenir
        # un jeu de paramÃ¨tres optimal.
        #Kgauss = cdist(X, X, lambda u, v: numpy.exp(-(numpy.linalg.norm(u - v)**2)) / (self.sigma**2))
        Kgauss = numpy.exp(-(cdist(X,X,'minkowski',p=2.)) / (self.sigma**2))
       
        def evaluateFunc(hypers):
            w0, *alphas = hypers
            h = lambda T : [numpy.sum(alphas*y*Kgauss[t],axis=0)+w0 for t in T]
            ind_mal_classe = numpy.where(y*h(list(range(len(X))))<1)[0]
            err = numpy.sum(1 - y[ind_mal_classe]*h(ind_mal_classe))+self.lambda_*numpy.sum(alphas)
            gradalpha = -y*numpy.dot(numpy.array(y[ind_mal_classe]),numpy.array(Kgauss[ind_mal_classe,:])) + self.lambda_
            gradW0 = [numpy.sum(y[ind_mal_classe])]
            grad = numpy.concatenate((gradW0,gradalpha))
            return err, grad
        """
        def evaluateFunc(hypers):
            w0, *alphas = hypers
            err = 0
            gradalpha = numpy.zeros(len(alphas))
            gradw0 = numpy.zeros(1)
            ind_mal_classe = []
            for t in range(len(y)):
                ht = numpy.sum([alphas[s]* y[s] * Kgauss[s,t] for s in range(len(X))]) + w0
                if (ht * y[t]) < 1:
                    ind_mal_classe.append(t)
                    gradw0 += y[t]
                    err += 1 - (y[t] * ht)
            
            for i in range(len(X)):
                gradalpha[i] = -numpy.sum([y[i]*y[t]*Kgauss[i,t] for t in ind_mal_classe])+self.lambda_
            err += self.lambda_ * numpy.sum(alphas)
            grad = numpy.concatenate((gradw0,gradalpha))
            return err, grad
        """
            
        # TODO Q3B
        # Initialisez aléatoirement les paramÃ¨tres alpha et omega0
        # (l'optimiseur requiert un "initial guess", et nous ne pouvons pas
        # simplement n'utiliser que des zéros pour différentes raisons).
        alpha = numpy.random.rand(X.shape[0])
        w0 = numpy.random.rand(1)
        # Stochez ces valeurs initiales aléatoires dans un array numpy nommé
        # "params"
        params = numpy.concatenate((w0,alpha),axis=0)        
        # Déterminez également les bornes Ã  utiliser sur ces paramÃ¨tres
        # et stockez les dans une variable nommée "bounds".
        # Indice : les paramÃ¨tres peuvent-ils avoir une valeur maximale (au-
        # dessus de laquelle ils ne veulent plus rien dire)? Une valeur
        # minimale? Référez-vous Ã  la documentation de fmin_l_bfgs_b
        # pour savoir comment indiquer l'absence de bornes.
        bounds = [(None,None)]+[(0,None) for i in range(len(alpha))]
        # Ã€ ce stade, trois choses devraient Ãªtre définies :
        # - Une fonction d'évaluation nommée evaluateFunc, capable de retourner
        #   l'erreur et le gradient d'erreur pour chaque paramÃ¨tre pour une
        #   configuration de paramÃ¨tres alpha et omega_0 donnée.
        # - Un tableau numpy nommé params de mÃªme taille que le nombre de
        #   paramÃ¨tres Ã  entraÃ®ner.
        # - Une liste nommée bounds contenant les bornes que l'optimiseur doit
        #   respecter pour chaque paramÃ¨tre
        # On appelle maintenant l'optimiseur avec ces informations et on stocke
        # les valeurs optimisées dans params
        #_times.append(time.time())
        params, minval, infos = fmin_l_bfgs_b(evaluateFunc, params, bounds=bounds)
        #_times.append(time.time())
        #checkTime(TMAX_FIT, "Entrainement")

        # On affiche quelques statistiques
        print("EntraÃ®nement terminé aprÃ¨s {it} itérations et "
              "{calls} appels Ã  evaluateFunc".format(it=infos['nit'], calls=infos['funcalls']))
        print("\tErreur minimale : {:.5f}".format(minval))
        print("\tL'algorithme a convergé" if infos['warnflag']
              == 0 else "\tL'algorithme n'a PAS convergé")
        print("\tGradients des paramÃ¨tres Ã  la convergence (ou Ã  l'épuisement des ressources) :")
        print(infos['grad'])

        # TODO Q3B
        # Stockez les paramÃ¨tres optimisés de la faÃ§on suivante
        # - Le vecteur alpha dans self.alphas
        # - Le biais omega0 dans self.w0
        self.alphas = params[1:]
        self.w0 = params[0]
        # On retient également le jeu d'entraÃ®nement, qui pourra
        # vous Ãªtre utile pour les autres fonctions Ã  implémenter
        self.X, self.y = X, y

    def predict(self, X):
        # TODO Q3B
        # Implémentez la fonction de prédiction
        # Vous pouvez supposer que fit() a préalablement été exécuté
        # et que les variables membres alphas, w0, X et y existent.
        # N'oubliez pas que ce classifieur doit retourner -1 ou 1
        Kgauss = cdist(X, X, lambda u, v: numpy.exp(-numpy.linalg.norm(u - v)**2) / (self.sigma**2))
        predictions = []
        for t in range(len(X)):
            ht = numpy.sum([y[s] * self.alphas[s] * Kgauss[s,t] for s in range(len(X))]) + self.w0
            predictions.append(int(numpy.sign(ht)))
        return predictions


    def score(self, X, y):
        # TODO Q3B
        # Implémentez la fonction retournant le score (accuracy)
        # du classifieur sur les données reÃ§ues en argument.
        # Vous pouvez supposer que fit() a préalablement été exécuté
        # Indice : réutiliser votre implémentation de predict() réduit de
        # beaucoup la taille de cette fonction!
        predictions = self.predict(X)
        return numpy.sum([predictions[i] == y[i] for i in range(len(y))]) / len(y)


In [ ]:
_times = []
_times.append(time.time())
# TODO Q3B
# CrÃ©ez le jeu de donnÃ©es Ã  partir de la fonction make_moons, tel que
# demandÃ© dans l'Ã©noncÃ©
# N'oubliez pas de vous assurer que les valeurs possibles de y sont
# bel et bien -1 et 1, et non 0 et 1!
X,y = make_moons(n_samples=1000, shuffle=True, noise=0.3, random_state=None)
y = (y * 2) - 1

# TODO Q3B
# SÃ©parez le jeu de donnÃ©es en deux parts Ã©gales, l'une pour l'entraÃ®nement
# et l'autre pour le test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=666, shuffle=True)
# TODO Q3B
# Une fois les paramÃ¨tres lambda et sigma de votre classifieur optimisÃ©s,
# crÃ©ez une instance de ce classifieur en utilisant ces paramÃ¨tres optimaux,
# et calculez sa performance sur le jeu de test.


#listLambda = [1,2.5,5]
listSigma = [0.1*x for x in range(1,10)]
listLambda = [1,5,10,25,50,100]
#listSigma = [0.1]
listDaN = []
ErrorDaN = {(l,s):0 for l in listLambda for s in listSigma}
for l in listLambda:
    for s in listSigma:
        listDaN.append(DiscriminantANoyau(lambda_ = l,sigma = s))
        
print("Nombre de classifieurs : ",len(listDaN))
for i in range(len(listDaN)):
    listDaN[i].fit(X_train, y_train)
    print("PARAM L & S",listDaN[i].lambda_, listDaN[i].sigma)
    print("Classificateur : ",i)
    ErrorDaN[listDaN[i].lambda_, listDaN[i].sigma] = 1 - listDaN[i].score(X_train, y_train)
    print("Error",ErrorDaN[listDaN[i].lambda_, listDaN[i].sigma])
minErrorDaN = min([ErrorDaN[i] for i in ErrorDaN.keys()])
argMinDaN = list(ErrorDaN.keys())[list(ErrorDaN.values()).index(minErrorDaN)]

print("argmin Lambda, Sigma = ",argMinDaN)
print("minErrorDaN", minErrorDaN)

Nombre de classifieurs :  54
EntraÃ®nement terminé aprÃ¨s 0 itérations et 21 appels Ã  evaluateFunc
	Erreur minimale : 528.33055
	L'algorithme n'a PAS convergé
	Gradients des paramÃ¨tres Ã  la convergence (ou Ã  l'épuisement des ressources) :
[-1.43000000e+02 -9.66062653e-05 -1.12414962e-04  9.97543429e-01
  9.99842136e-01 -2.77715739e-09  3.20527825e-06 -1.75629448e-01
 -5.88012648e-03 -3.24132175e-02  9.99984411e-01  4.17919276e-04
 -3.60197250e-03 -1.51248935e-04 -8.44470157e-04  9.99999996e-01
 -5.26694685e-03 -3.06834552e-06  2.10666853e-02 -3.49481710e-09
 -8.14856238e-06 -4.27278881e-01  9.99921715e-01  5.54369303e-04
 -1.45368618e-03  9.99951585e-01 -2.44769226e-05 -3.86782483e-03
  9.99989613e-01  1.00451372e+00  9.99697627e-01  9.99999923e-01
 -1.40563927e-02  9.75393505e-01  9.99999546e-01 -6.41214861e-02
  1.20382369e-02 -5.82061639e-04 -2.37558957e-02  1.00000609e+00
 -2.39354728e-05 -1.19452190e-05  9.99965145e-01  1.08592252e-05
  1.00000000e+00 -7.37666039e-03 -1.280753

Error 0.29800000000000004
EntraÃ®nement terminé aprÃ¨s 0 itérations et 21 appels Ã  evaluateFunc
	Erreur minimale : 486.83416
	L'algorithme n'a PAS convergé
	Gradients des paramÃ¨tres Ã  la convergence (ou Ã  l'épuisement des ressources) :
[-2.04000000e+02  9.94753216e-01 -1.80189925e-01  1.24393553e+00
  1.02692563e+00  9.98238769e-01  1.05315197e+00  5.46820002e-02
 -9.79568962e-01 -1.16882029e+00  9.85491487e-01  1.45470906e+00
 -5.21132270e-01 -4.55440644e-01 -5.16739617e-01 -5.25774876e-01
  1.15467153e+00  9.40218546e-01  7.40610995e-01  9.92816510e-01
 -2.07586236e-01 -2.60590990e+00  1.03261518e+00  1.30738823e-01
 -4.53505514e-02  9.99216006e-01 -1.36526764e-01 -3.96334079e-01
 -6.61318433e-03  1.33385949e+00  9.99824053e-01  9.98029804e-01
 -7.75310393e-01  9.66426497e-01  9.96010032e-01 -8.64381167e-01
  2.31507835e-01 -1.65861388e-01 -5.46362064e-01  1.12873619e+00
 -2.04379847e-01  9.32708765e-01  9.69314976e-01 -1.45566435e-03
  9.99999261e-01 -5.69659832e-01  9.99924556e

Error 0.23399999999999999
EntraÃ®nement terminé aprÃ¨s 1 itérations et 29 appels Ã  evaluateFunc
	Erreur minimale : 340.72641
	L'algorithme n'a PAS convergé
	Gradients des paramÃ¨tres Ã  la convergence (ou Ã  l'épuisement des ressources) :
[-2.40000000e+01  8.62122743e-01  7.42365235e-01  1.83564217e+00
  1.34247827e+00  9.21867825e-01  4.68788115e-01  1.07930524e+00
  9.75162914e-01  1.77385484e+00  1.07152351e+00  1.27908844e-01
 -1.53072204e-01  7.82383973e-01  4.26487549e-01  8.27626594e-01
  1.52969181e+00  9.69933833e-01  1.50292697e+00 -3.94460870e-02
  8.91762577e-01  9.20189220e-01  1.45470215e+00  4.97936249e-01
  2.26703566e-01  1.09627401e+00 -5.13150996e-01 -3.98555719e-01
  9.44928258e-01  4.57641695e-02  1.01314168e+00  9.45830734e-01
  6.28894500e-01  9.45442718e-01  9.31583707e-01  8.89329812e-01
  5.94251096e-01 -1.55629031e-01  1.04343390e+00  6.92790108e-01
  9.94060855e-01  9.85494559e-01  9.98741550e-01  6.98105525e-01
 -1.66123559e-01 -4.63626439e-01  7.75543268e

Error 0.25
EntraÃ®nement terminé aprÃ¨s 1 itérations et 29 appels Ã  evaluateFunc
	Erreur minimale : 401.60206
	L'algorithme n'a PAS convergé
	Gradients des paramÃ¨tres Ã  la convergence (ou Ã  l'épuisement des ressources) :
[-4.20000000e+01  1.01882489e+00 -9.35242628e-01  3.56831584e+00
  2.36858372e+00  1.14036464e+00  1.99692488e+00  1.54987318e+00
  6.46839838e-01  2.36377231e+00  1.72396595e+00 -2.30255943e-01
  3.55445001e-01  7.16004779e-01  3.10790338e-01  1.00399924e+00
  2.51323287e+00  1.10794266e+00  4.68654471e+00  1.01000789e+00
  4.05202326e-01  6.70674853e-01  2.87807709e+00  1.15655994e+00
  3.82846809e-02  1.69013317e+00 -2.34104783e+00 -2.57078229e+00
  1.13528474e+00  2.64265993e+00  1.44181198e+00  1.03666267e+00
  4.03308160e-01  1.05809071e+00  1.03994621e+00  1.02682238e+00
  2.82227368e+00 -9.02513411e-01  1.30502318e+00  1.88166488e+00
  2.73824580e-01  1.10714474e+00  1.19766384e+00 -5.37949848e-01
  1.00107365e+00  1.50528168e+00  1.01250869e+00  3.38681286

Error 0.15200000000000002
EntraÃ®nement terminé aprÃ¨s 4 itérations et 48 appels Ã  evaluateFunc
	Erreur minimale : 301.52640
	L'algorithme n'a PAS convergé
	Gradients des paramÃ¨tres Ã  la convergence (ou Ã  l'épuisement des ressources) :
[-4.00000000e+01  1.10552129e+00 -1.30916430e-01  2.30396472e+00
  2.21055706e+00  1.55232380e+00  2.93748513e+00 -1.05548950e+00
 -4.17973274e-01 -2.32602147e+00  1.96687921e+00  3.71579316e+00
 -1.58135016e-01 -2.01386447e-01 -1.21237001e+00  1.04252225e+00
  3.99366633e+00  1.25545653e+00  1.74206931e+00  1.05864812e+00
 -9.05464321e-03 -2.15016247e-01  2.08683527e+00  1.06624119e+00
  3.00514303e-01  1.80675089e+00 -3.42568580e-01 -7.57871856e-01
  1.32896932e+00  4.22383771e+00  1.54824584e+00  1.21086936e+00
 -1.09462597e+00  1.17208980e+00  1.19625903e+00 -5.61162390e-01
 -4.13637479e+00 -1.71882043e+00 -1.23422640e+00  4.17035868e-01
 -1.59474804e+00  1.41781634e+00  1.37861244e+00 -2.75834991e+00
  1.01512570e+00 -4.24164654e+00  1.10932768e

Error 0.23199999999999998


In [ ]:
fig, sfig = pyplot.subplots(1, 1, sharex=True, sharey=True)
err = sfig.scatter([list(ErrorDaN.keys())[i][0] for i in range(len(list(ErrorDaN.keys())))],
             [list(ErrorDaN.keys())[i][1] for i in range(len(list(ErrorDaN.keys())))], 
             s=80, c=list(ErrorDaN.values()))
sfig.set_xlabel("lambda")
sfig.set_ylabel("Sigma")
pyplot.colorbar(err, format='Err %.2f')
pyplot.show()

In [ ]:
# TODO Q3B
# CrÃ©ez ici une grille permettant d'afficher les rÃ©gions de
# dÃ©cision pour chaque classifieur
# Indice : numpy.meshgrid pourrait vous Ãªtre utile ici
# Par la suite, affichez les rÃ©gions de dÃ©cision dans la mÃªme figure
# que les donnÃ©es de test.
# Note : utilisez un pas de 0.02 pour le meshgrid

#pas = 100
#x1 = np.linspace(X[:, 0].min(), X[:, 0].max(), pas)
#x2 = np.linspace(X[:, 1].min(), X[:, 1].max(), pas)
pas = 0.2
x1 = numpy.arange(X[:, 0].min(), X[:, 0].max(), pas)
x2 = numpy.arange(X[:, 1].min(), X[:, 1].max(), pas)
absc, ordon = numpy.meshgrid(x1, x2)

lambda_opti, sigma_opti = argMinDaN
clfopti = DiscriminantANoyau(lambda_opti, sigma_opti)
clfopti.fit(X_test, y_test)

print("Score DiscLinGauss: ", clfopti.score(X_test, y_test))
fig, sfig = pyplot.subplots(1, 1, sharex=True, sharey=True)
colors = ["bgr"[i] for i in clfopti.predict(numpy.c_[absc.ravel(), ordon.ravel()])]
plop = numpy.array(clfopti.predict(numpy.c_[absc.ravel(), ordon.ravel()]))
plop = plop.reshape(absc.shape)
#sfig.contourf(absc,ordon,plop, cmap=pyplot.cm.Paired, alpha=0.8)
sfig.scatter(absc, ordon, alpha=0.5, s=20, c=colors)
#sfig.scatter(X_train[:, 0], X_train[:, 1], marker='*',c=["bgr"[i] for i in y_train],alpha=0.2)
sfig.scatter(X_test[:,0], X_test[:,1], marker='s',cmap=pyplot.cm.Paired, color=["bgr"[i] for i in y_test], alpha=0.5)
sfig.set_title("Erreur : " + str("%.2f" % round(1 - clfopti.score(X_test, y_test), 2)),fontsize=20)
pyplot.show()
# filled_markers = ('o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X')

2.5 0.5

In [ ]:
X,y = make_moons(n_samples=1000, shuffle=True, noise=0.3, random_state=None)
y = (y * 2) - 1

# TODO Q3B
# SÃ©parez le jeu de donnÃ©es en deux parts Ã©gales, l'une pour l'entraÃ®nement
# et l'autre pour le test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=666, shuffle=True)


In [ ]:
Kgauss = cdist(X_train[:5], X_train[:5], 'minkowski')

In [ ]:
Kgauss

In [ ]:
Kgauss[:,[1,3,4]]

In [ ]:
plop = numpy.sum(numpy.array([1,-1,1])*numpy.array(Kgauss[:,[1,3,4]]), axis=1)
plop

In [ ]:
numpy.dot(numpy.array([1,-1,1]),numpy.array(Kgauss[[1,3,4],:]))

In [ ]:
plop = numpy.dot(numpy.array([1,-1,1]),numpy.array(Kgauss[[1,3,4],:]))
plop

In [ ]:
y[:5]*plop + 1

In [ ]:
numpy.sum(numpy.array([1,-1,1]).reshape(3,1)*numpy.array(Kgauss[[1,3,4],:]), axis=1)

In [ ]:
alpha = numpy.random.rand(X.shape[0])